# Chatbot Implementation with Reformer
---

last updated: Nov 08, 2020

### References

This notebook was learned and modified from the assignment of the course [Natural Language Processing with Attention Models](https://www.coursera.org/learn/attention-models-in-nlp) on *Coursera* with the following amendments:

1. Instead of using a pre-trained model, I trained the chatbot **from scratch**

2. Instead of using `trax.supervised.decoding.autoregressive_sample_stream`, I used a helper function to continue the conversation token by token.

3. I cleaned up and rewrited the part of model evaluation. The model can be used to predict on the test dataset, or any custom starter sentence.
---

### Project Summary

1. After training on the MultiWOZ dataset (trainning size = 9,938) for 20,000 epochs, the model achieved the **accuracy of 67.15%** on the test set (size = 500).

2. By feeding in some custom stencences, the **chatbot is able to handle the conversation and address the request**. For example conversations, please go to *Step 6-4-3* (at the end of this notebook).

---
### The Reformer Paper

N. Kitaev et al., **Reformer: The Efficient Transformer**

https://arxiv.org/abs/2001.04451

---

## Step 1: Install prerequisites

### 1-1 Install Trax

In [1]:
!pip -q install trax

### 1-2 Import libraries

In [2]:
import json
import random
import numpy as np

import trax
from trax import layers as tl
from trax.supervised import training

from termcolor import colored

## Step 2: Download and Upload the Dataset

---

**Whcih dataset?**

Datasets of dialogues can be chosen from the following website:

https://lionbridge.ai/datasets/15-best-chatbot-datasets-for-machine-learning/

This is the dataset I used:


**MultiWOZ: Multi-Domain Wizard-of-Oz dataset (ver 2.1)**

http://dialogue.mi.eng.cam.ac.uk/index.php/corpus/

### 2-1 Download the dataset

Download the dataset `data.json` (263MB) from the website above

### 2-2 Upload the dataset

Upload `data.json` to the current directory. It may take a while.

### 2-3 Load the dataset

In [3]:
with open('data.json') as file:
    multiwoz_dataset_json = json.load(file)

### 2-4 Inspect the data format

In [4]:
print("Dataset loaded. Number of dialogues: {}".format(len(multiwoz_dataset_json)))

Dataset loaded. Number of dialogues: 10438


In [5]:
print(multiwoz_dataset_json.keys())

dict_keys(['SNG01856.json', 'SNG0129.json', 'PMUL1635.json', 'MUL2168.json', 'SNG0073.json', 'SNG01445.json', 'MUL2105.json', 'PMUL1690.json', 'MUL2395.json', 'SNG0190.json', 'PMUL1170.json', 'SNG01741.json', 'PMUL4899.json', 'MUL2261.json', 'SSNG0348.json', 'MUL0784.json', 'MUL0886.json', 'PMUL2512.json', 'SNG0548.json', 'MUL1474.json', 'PMUL4372.json', 'PMUL4047.json', 'PMUL1181.json', 'PMUL0287.json', 'PMUL3470.json', 'PMUL0151.json', 'MUL0586.json', 'PMUL3552.json', 'PMUL1539.json', 'MUL1790.json', 'PMUL3021.json', 'SNG0699.json', 'SNG0228.json', 'PMUL3296.json', 'MUL1434.json', 'PMUL2203.json', 'PMUL3250.json', 'PMUL0510.json', 'MUL1124.json', 'PMUL3719.json', 'PMUL4648.json', 'PMUL2437.json', 'SNG0297.json', 'PMUL2049.json', 'SNG01722.json', 'PMUL2100.json', 'MUL1853.json', 'MUL2694.json', 'SNG1006.json', 'SNG1345.json', 'MUL1299.json', 'MUL1490.json', 'PMUL2749.json', 'PMUL2804.json', 'MUL1628.json', 'PMUL2202.json', 'SNG01450.json', 'SNG0131.json', 'SNG0984.json', 'PMUL1419.jso

In [6]:
print(multiwoz_dataset_json['SNG01856.json'].keys())

dict_keys(['goal', 'log'])


In [7]:
print(multiwoz_dataset_json['SNG01856.json']['log'])

[{'text': 'am looking for a place to to stay that has cheap price range it should be in a type of hotel', 'metadata': {}}, {'text': 'Okay, do you have a specific area you want to stay in?', 'metadata': {'taxi': {'book': {'booked': []}, 'semi': {'leaveAt': '', 'destination': '', 'departure': '', 'arriveBy': ''}}, 'police': {'book': {'booked': []}, 'semi': {}}, 'restaurant': {'book': {'booked': [], 'time': '', 'day': '', 'people': ''}, 'semi': {'food': '', 'pricerange': '', 'name': '', 'area': ''}}, 'hospital': {'book': {'booked': []}, 'semi': {'department': ''}}, 'hotel': {'book': {'booked': [], 'stay': '', 'day': '', 'people': ''}, 'semi': {'name': 'not mentioned', 'area': 'not mentioned', 'parking': 'not mentioned', 'pricerange': 'cheap', 'stars': 'not mentioned', 'internet': 'not mentioned', 'type': 'hotel'}}, 'attraction': {'book': {'booked': []}, 'semi': {'type': '', 'name': '', 'area': ''}}, 'train': {'book': {'booked': [], 'people': ''}, 'semi': {'leaveAt': '', 'destination': '',

In [8]:
## Displayed a dialogue with colors

sample_dialogue = multiwoz_dataset_json['SNG0129.json']['log']

for i in range(len(sample_dialogue)):
    if i % 2 == 0:
        print(colored(sample_dialogue[i]['text'], 'blue'))
    else:
        print(colored(sample_dialogue[i]['text'], 'red'))


Hello, I have been robbed.  Can you please help me get in touch with the police?
Parkside Police Station is in Parkside, Cambridge. Their number is 01223358966. Anything else I can do for you?
Can I please have the postcode as well?
The postcode for the Parkside Police Station is CB11JG. Can I help you with anything else?
Was Parkside the address of the police station? If not, can I have the address please?
Yes, Parkside is the address.
Thank you that will be all for now.
Great. Thank you for contacting Cambridge Towninfo Centre.
You were great. Goodbye.
We are happy to help. Have a good day!


## Step 3: Data Preprocessing

### 3-1 Add "Person 1" and "Person 2" into the dialogue

In [9]:
dialogue_sentences_list = []

for json_index in multiwoz_dataset_json.keys():
    
    dialogue = multiwoz_dataset_json[json_index]['log']

    dialogue_sentences_str = ""

    for i in range(len(dialogue)):

        if i % 2 == 0:
            dialogue_sentences_str += " Person 1: " + dialogue[i]['text']
        else:
            dialogue_sentences_str += " Person 2: " + dialogue[i]['text']
    
    dialogue_sentences_list.append(dialogue_sentences_str)

In [10]:
print(len(dialogue_sentences_list))

10438


In [11]:
print(dialogue_sentences_list[0])

 Person 1: am looking for a place to to stay that has cheap price range it should be in a type of hotel Person 2: Okay, do you have a specific area you want to stay in? Person 1: no, i just need to make sure it's cheap. oh, and i need parking Person 2: I found 1 cheap hotel for you that includes parking. Do you like me to book it? Person 1: Yes, please. 6 people 3 nights starting on tuesday. Person 2: I am sorry but I wasn't able to book that for you for Tuesday. Is there another day you would like to stay or perhaps a shorter stay? Person 1: how about only 2 nights. Person 2: Booking was successful.
Reference number is : 7GAWK763. Anything else I can do for you? Person 1: No, that will be all. Good bye. Person 2: Thank you for using our services.


### 3-2 Shuffle the dataset

In [12]:
## shuffle the list
random.shuffle(dialogue_sentences_list)

### 3-3 Split into training and test data

In [14]:
## Split 500 dialogues to the test dataset
train_data, test_data = dialogue_sentences_list[:-500], dialogue_sentences_list[-500:]

print("Number of train_data: {}".format(len(train_data)))
print("Number of test_data: {}".format(len(test_data)))

Number of train_data: 9938
Number of test_data: 500


In [15]:
print(train_data[0])

 Person 1: I am looking for places to go in the north area. Person 2: I have a boat, park or 2 swimming pools in that area. Is there one you would like to try? Person 1: Give me more information about the boat please. Person 2: The address to riverboat georgina is cambridge passenger cruisers, jubilee house and the postcode is cb43ax. Can I help with anything else? Person 1: I'm looking for a place to eat that is in the same area. I want it to be cheap.  Person 2: You've got 2 choices in the north for cheap places to eat: Royal Spice serves Indian food, and Da Vinci Pizzeria serves Italian food. Which would you prefer? Person 1: Da Vinci Pizzeria sounds good.  Thanks. Person 2: For how many do you need this reservation? Person 1: I need to book a table for 8 at 18:30 on Monday, please. Person 2: Your table is reserved at DaVinci, ref# 6U1JO0B2. Is there anything else I can do for you today? Person 1: I also need a taxi that will get me  to the restaurant in time for the booking you jus

In [16]:
print(test_data[0])

 Person 1: Hello, do you have any information regarding interesting theatres to visit in the centre of town? Person 2: Sure thing, there are four theatres in the centre of town. ADC Theatre, Cambridge Arts Theatre, Mumford Theatre, and The Cambridge Corn Exchange.  Person 1: I'm interested in one of the theatres! Can I get the entrance fee information for those? Person 2: I'm sorry, but none of the theaters list the entrance fee. Would you like the phone number for any of them? Person 1: That's okay, could you provide me with the postcode and address, please? Also, I am looking for a train leaving after 16:00 this Monday. Person 2: The ADC Theatre is located on Park Street and its postcode is cb23pj. Before I find your train, could you tell me where you would like to go? Person 1: Of course, silly me :P I am looking for a train from cambridge to stevenage Person 2: The TR0254 leaves Cambridge at 17:21. Person 1: Awesome, thank you very much! I'm pretty sure that's all I need right now,

### 3-4 Remove leading or ending white spaces

In [17]:
for i in range(len(train_data)):
    train_data[i] = train_data[i].strip()

for i in range(len(test_data)):
    test_data[i] = test_data[i].strip()

### 3-5 Data pipeline

In [18]:
def stream_generator(data):
    while True:
        x = random.choice(data)
        yield (x, x)

In [19]:
data_pipeline = trax.data.Serial(trax.data.Shuffle(),
                                 trax.data.Tokenize(vocab_file = 'en_32k.subword'),
                                 trax.data.FilterByLength(2048),
                                 trax.data.BucketByLength(boundaries = [128, 256, 512, 1024], batch_sizes = [16, 8, 4, 2, 1]),
                                 trax.data.AddLossWeights(id_to_mask = 0))

train_stream = data_pipeline(stream_generator(train_data))
test_stream = data_pipeline(stream_generator(test_data))

In [20]:
## Check 1
# (input, target, weights)

print("train_stream")
print(next(train_stream))
print("\ntest_stream")
print(next(test_stream))

train_stream
(array([[8745,    3,   54, ...,    0,    0,    0],
       [8745,    3,   54, ...,    0,    0,    0],
       [8745,    3,   54, ...,    0,    0,    0],
       [8745,    3,   54, ...,    0,    0,    0]]), array([[8745,    3,   54, ...,    0,    0,    0],
       [8745,    3,   54, ...,    0,    0,    0],
       [8745,    3,   54, ...,    0,    0,    0],
       [8745,    3,   54, ...,    0,    0,    0]]), array([[1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.],
       [1., 1., 1., ..., 0., 0., 0.]], dtype=float32))

test_stream
(array([[8745,    3,   54, ...,    0,    0,    0],
       [8745,    3,   54, ...,    0,    0,    0],
       [8745,    3,   54, ...,    0,    0,    0],
       [8745,    3,   54, ...,    0,    0,    0]]), array([[8745,    3,   54, ...,    0,    0,    0],
       [8745,    3,   54, ...,    0,    0,    0],
       [8745,    3,   54, ...,    0,    0,    0],
       [8745,    3,   54, ...,    0,    0,    0

In [21]:
## Check

x = next(train_stream)[0]
y = next(test_stream)

print(x.shape)
print(x)

y = trax.data.detokenize(x[0], vocab_file = 'en_32k.subword')

print(y)

del x, y

(4, 512)
[[8745    3   54 ...    0    0    0]
 [8745    3   54 ...    0    0    0]
 [8745    3   54 ...    0    0    0]
 [8745    3   54 ...    0    0    0]]
Person 1: Please dig up some information on the mutliple sports in the centre for me Person 2: I'm afraid there isn't one. Can I help you find something else? Person 1: Oh well. I guess maybe a theatre instead.  Person 2: There are 4 theatres in the centre. They are adc theatre, cambridge arts theatre, mumford theatre, and the cambridge corn exchange. Person 1: Can you tell me the postcode, entrance fee, and phone number of adc theatre? Person 2: The entrance fee isn't listed, but you can reach them by phone at 01223300085. The postcode is cb58as. Person 1: Thanks I also need a train that's going to cambridge.  Person 2: No problem, where will that be coming from? Person 1: I need to leave from Norwich sometime after 20:15 on Saturday. Person 2: TR0615 departs at 20:16 and arrives by 21:35. Would you like a ticket? Person 1: No.  

## Step 4: Create the Reformer model

**Trax Documentation**

[trax.models.reformer.ReformerLM](https://trax-ml.readthedocs.io/en/latest/trax.models.html#trax.models.reformer.reformer.ReformerLM)

This model only uses the decoder (no encoder).


In [23]:
def ReformerLM(vocab_size = 33000, n_layers = 6, mode = 'train', attention_type = tl.SelfAttention):

    """
    Create the Reformer

    Inputs
            vocab_size: <int> the size of the vocabulary
            n_layers: <int> number of decoder layers
            mode: <str> 'train', 'eval', or 'predict'
            attention_type: <trax class> type of the attention class

    Output
            model: <trax model> the Reformer model
    """

    model = trax.models.reformer.ReformerLM(vocab_size = vocab_size,
                                            n_layers = n_layers,
                                            mode = mode,
                                            attention_type = attention_type)

    return model

## Check
model = ReformerLM(mode = 'train')
print(str(model))
del model 

Serial[
  Serial[
    AssertShape
    ShiftRight(1)
    AssertShape
  ]
  Embedding_33000_512
  Dropout
  PositionalEncoding
  Dup_out2
  ReversibleSerial_in2_out2[
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        FastGelu
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        FastGelu
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidual_in2_out2[
      Seria

## Step 5: Train the Reformer

### 5-1 The training loop

In [24]:
def training_loop(ReformerLM, train_generator, eval_generator):

    """
    Create the training loop

    Inputs
            ReformerLM: <Trax model> the Reformer model
            train_generator: <Trax stream> the generator stream of training data
            eval_generator: <Trax stream> the generator stream of the test data
    
    Output
            loop: <Trax loop object> the loop object
    """

    # schedule of the learning rate
    lr_schedule = trax.lr.warmup_and_rsqrt_decay(n_warmup_steps = 4000, max_value = 0.001)

    # the training task
    train_task = training.TrainTask(labeled_data = train_generator,
                                    loss_layer = tl.CrossEntropyLoss(),
                                    optimizer = trax.optimizers.Adam(0.001),
                                    lr_schedule = lr_schedule,
                                    n_steps_per_checkpoint = 200)
    
    # the evaluation task
    eval_task = training.EvalTask(labeled_data = eval_generator,
                                  metrics = [tl.CrossEntropyLoss(), tl.Accuracy()])
    
    # create the loop object
    loop = training.Loop(model = ReformerLM(mode = 'train'),
                         tasks = [train_task],
                         eval_tasks = [eval_task],
                         output_dir = '.')
    
    return loop

### 5-2 Train the model


In [25]:
## This box needs to be executed if training_loop has been run before
!rm model.pkl.gz
!rm config.gin
!rm -r train
!rm -r eval

rm: cannot remove 'model.pkl.gz': No such file or directory
rm: cannot remove 'config.gin': No such file or directory
rm: cannot remove 'train': No such file or directory
rm: cannot remove 'eval': No such file or directory


In [26]:
loop = training_loop(ReformerLM, train_stream, test_stream)
loop.run(20000)


Step      1: Total number of trainable weights: 70673640
Step      1: Ran 1 train steps in 60.75 secs
Step      1: train CrossEntropyLoss |  10.44453430
Step      1: eval  CrossEntropyLoss |  10.43628788
Step      1: eval          Accuracy |  0.00000000

Step    200: Ran 199 train steps in 131.59 secs
Step    200: train CrossEntropyLoss |  8.29019642
Step    200: eval  CrossEntropyLoss |  5.70468712
Step    200: eval          Accuracy |  0.07524613

Step    400: Ran 200 train steps in 79.36 secs
Step    400: train CrossEntropyLoss |  5.62913370
Step    400: eval  CrossEntropyLoss |  5.53083134
Step    400: eval          Accuracy |  0.06506850

Step    600: Ran 200 train steps in 60.35 secs
Step    600: train CrossEntropyLoss |  5.52661419
Step    600: eval  CrossEntropyLoss |  5.57809830
Step    600: eval          Accuracy |  0.07159904

Step    800: Ran 200 train steps in 60.02 secs
Step    800: train CrossEntropyLoss |  4.99216938
Step    800: eval  CrossEntropyLoss |  4.53790379
St

## Step 6: Model Evaluation

### 6-1 Helper functions to tokenize/detokenize

In [27]:
def tokenize(sentence):
    return list(trax.data.tokenize(iter([sentence]), vocab_file = 'en_32k.subword'))[0]

def detokenize(tokens):
    return trax.data.detokenize(tokens, vocab_file = 'en_32k.subword')

### 6-2 Helper function to print out the dialogue in color

In [28]:
## The helper function to print out the dialogues in colors

def print_colored_dialogue(dialogues):

    result = []

    cur_conversation = ""

    first_sentence_printed = False

    Person1_turn = True

    for s in dialogues:

        cur_conversation += s

        # model predicts Person 2 finishes the sentence
        if cur_conversation.endswith("Person 1: "):
            if not first_sentence_printed:
                first_sentence_printed = True
            else:
                # print everything before "Person 1: "
                print(colored("Person 2: " + cur_conversation.split("Person 1: ")[0].strip(), 'red'))
                cur_conversation = ""
                Person1_turn = True

        # model predicts Person 1 finished the sentence
        elif cur_conversation.endswith("Person 2: "):
            # print everything before "Person 2: "
            print(colored("Person 1: " + cur_conversation.split("Person 2: ")[0].strip(), 'blue'))
            cur_conversation = ""
            Person1_turn = False

    # print remaining sentences
    if Person1_turn:
        print(colored("Person 1: " + cur_conversation, 'blue'))
    else:
        print(colored("Person 2: " + cur_conversation, 'red'))

### 6-3 Predict on the test dataset

In [29]:
# grab a batch from test_stream
test_x, test_y, test_w = next(test_stream)
print("Batch_size = {}".format(test_x.shape[0]))

# choose the first example
sample_x = test_x[0][None, :]

print("\nInput dialogue:")
print_colored_dialogue(detokenize(sample_x[0]))

pred = loop.eval_model(sample_x)
pred_token = pred.argmax(axis = -1)

print("\nOutput dialogue:")
print_colored_dialogue(detokenize(pred_token[0]))

Batch_size = 4
Input dialogue:
Person 1: Person 1: I am  looking for a place to stay. The hotel should be in the type of guesthouse and should include free wifi
Person 2: There are 23 guesthouses with free wifi. What price range or area do you prefer?
Person 1: I would prefer a price range of expensive
Person 2: It seems that there are no guesthouses in the expensive price range. Can I offer you a different price range or a different type of hotel?
Person 1: How about moderate price range with a star of 4?
Person 2: There are several. Do you have a preference for area?
Person 1: Area doesn't matter but I need to know if they have free parking
Person 2: I have discovered 9 choices fitting your description. May I recommend the Archway House? It's on Gilbert Road.
Person 1: Sounds great, Thank you.
Person 2: You're welcome. Would you like me to make a reservation in your name?
Person 1: Yes please make a reservation in my name. Thank you.
Person 2: I will need to know the day you will be 

### 6-4 Predict on custom starter sentence

6-4-1 Helper function to generate the next token

In [50]:
def generate_next_token(current_tokens, model):

    """
    Generate the next token
    
    Inputs
            current_tokens: <list of int> currently generated token so far
            model: <trax model> the model for the prediction
    
    Output
            next_token: <int> the next token generated by the model
    """

    # number of tokens generated so far
    current_tokens_length = len(current_tokens)
   
    # find the next power of 2 to be the final length after padding
    final_padded_length = 2**int(np.ceil(np.log2(current_tokens_length + 1)))

    # caucluate the number of zeros to pad
    to_pad_length = final_padded_length - current_tokens_length

    # padding
    padded_current_tokens = np.array(current_tokens.tolist() + [0 for _ in range(to_pad_length)])[None, :]

    # use the model to predict the log probabilities of the next token
    model_output, _ = model((padded_current_tokens, padded_current_tokens))

    # (note) model_output has shape (batch_size, len_of_whole_token_list, vocab_size)
    # only take the log probability distribution of the last token
    next_token_logprob = model_output[0, current_tokens_length, :]

    # select the token with the largest log probability
    next_token = int(np.argmax(next_token_logprob))

    return next_token

#### 6-4-2 Helper function to continue the conversation token by token

In [53]:
def extend_dialogue(current_dialogue, model, maximum_number_extension = 100):

    """
    Extend the dialogue

    Inputs 
            current_dialogue: <str> the starting dialogue
            model: <trax model> the chatbot model
            maximum_number_extension: <int> maximum number of tokens to generate

    Output
            complete_dialogue: <str> the complete dialogue
    """

    current_tokens_list = tokenize(current_dialogue)

    num_tokens_generated = 0

    while num_tokens_generated <= maximum_number_extension:
        # given current_tokens_list, generate the next token
        next_output_token = generate_next_token(current_tokens_list, model)

        current_tokens_list = current_tokens_list.tolist()

        current_tokens_list.append(next_output_token)

        current_tokens_list = np.array(current_tokens_list)

        num_tokens_generated += 1

    # maximum number of tokens reached, output the detokenized dialogue
    complete_dialogue = trax.data.detokenize(current_tokens_list, vocab_file = 'en_32k.subword')

    return complete_dialogue

#### 6-4-3 Custom starter sentences

Use and starter sentences as `dialogue_seed`. However, it has to begin with "Person 1:" and end with "Person 2: "

In [55]:
## Example wehre Person 1 asks for an avocado for no reason

dialogue_seed = "Person 1: Um... Can I have some avocado? Person 2: "

complete_dialogue = extend_dialogue(dialogue_seed, loop.eval_model, maximum_number_extension = 100)

print_colored_dialogue(complete_dialogue)

Person 1: Person 1: Um... Can I have some avocado?
Person 2: I can't help you with that. What type of information are you looking for?
Person 1: I'm looking for a train to Cambridge that leaves after 15:00.
Person 2: I have train TR7424 that leaves at 17:11 and arrive in Cambridge at 18:55. Would you like me to book it for you?
Person 1: Yes, I need 5 tickets.
Person 2: Booking was successful, the total fee is


In [56]:
## Example where Person 1 wants to find a restaurant

dialogue_seed = "Person 1: Where is the Japanese restaurant? Person 2: "

complete_dialogue = extend_dialogue(dialogue_seed, loop.eval_model, maximum_number_extension = 100)

print_colored_dialogue(complete_dialogue)

Person 1: Person 1: Where is the Japanese restaurant?
Person 2: The Addenbell is located at 32 Bridge Street City Centre. Would you like me to book a table?
Person 1: No, I'd like to book a table for 7 people at 14:00 on Saturday.
Person 2: I have booked you a table for 7 at the Bedouin. Your reference number is XYZ1Z.
Person 1: Thanks, that's all I need.
Person 2: You're welcome


In [58]:
## Example where Person 1 wants to find a post office

dialogue_seed = "Person 1: Where can I find the post office? Person 2: "

complete_dialogue = extend_dialogue(dialogue_seed, loop.eval_model, maximum_number_extension = 100)

print_colored_dialogue(complete_dialogue)

Person 1: Person 1: Where can I find the post office?
Person 2: The Cambridge Belfry is located at 5 Greens Road.
Person 1: What is the postcode?
Person 2: The postcode is cb21sj.
Person 1: Thanks, that's all I need.
Person 2: You're welcome. Have a great day!and enjoy your day!us.
Person 1: Thanks, you too.
Person 2: You're welcome. Have a great day!.Person 1


In [60]:
## Example where Person 1 wants book a hotel room

dialogue_seed = "Person 1: Hello, I'm wondering if I can book a hotel room tonight? Person 2: "

complete_dialogue = extend_dialogue(dialogue_seed, loop.eval_model, maximum_number_extension = 200)

print_colored_dialogue(complete_dialogue)

Person 1: Person 1: Hello, I'm wondering if I can book a hotel room tonight?
Person 2: I can help with that. What area of town would you like to stay in?
Person 1: I'd like to stay in the east, please.
Person 2: I have three options for you. The Huntingdon Marriott Hotel, and the rest of your stay.
Person 1: I'd like to book it for 2 people and 4 nights starting from wednesday.
Person 2: I have booked your hotel for 4 people for 2 nights starting on Saturday. Your reference number is X1Z1Z1.
Person 1: Thanks, that's all I need.
Person 2: You're welcome. Have a great day!!!!!!.
Person 1: Thank you, goodbye.
Person 2: Thank you for using our service. Have a great day!.
Person 1: Thanks, you too.
Person 2: You're welcome. 
